In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

True

In [ ]:
import pandas as pd
val_df = pd.read_csv("workshop_data/rt_val_set.csv", on_bad_lines='skip', engine="python")
train_df = pd.read_csv("workshop_data/rt_train_set.csv", on_bad_lines='skip', engine="python")
test_df = pd.read_csv("workshop_data/rt_test.csv", on_bad_lines='skip', engine="python")
print("#Train = " + str(len(train_df)))
print("#Val = " + str(len(val_df)))
print("#Test = " + str(len(test_df)))
test_df.head(5)

#Train = 7035
#Val = 449
#Test = 3178


,Unnamed: 0,review,sentiment
0,0,plotless collection of moronic stunts is by fa...,0
1,1,plays like some corny television production fr...,0
2,2,what with all the blanket statements and dime-...,0
3,3,"according to wendigo , 'nature' loves the memb...",0
4,4,"on its own , it's not very interesting . as a ...",0


In [ ]:
ADADELATA_OPT = 'adadelta'
ADAM_OPT = 'adam'
BATCH_SIZE = "batch_size"
BCE_LOSS = 'bce_loss'
CNN_KERNELS = "cnn_kernels"
CNN_OUT_CHANNELS = "cnn_out_channels"
CROSS_ENTROP_LOSS = 'cross_entropy_loss'
DROPOUT = "dropout"
EARLY_STOP_MAX_NO_IMP = "early_stop_max_no_imp"
EMBED_WORDS_FILE = "embed_words_file"
FREEZE_EMBEDDING = "freeze_embeding"
LABEL_COL = "label_col"
LEARNING_RATE = "learning_rate"
LENGTH_COL = "length_col"
LOG_FILE_NAME = "log_file_name"
LOSS_FUNCTION = "loss_function"
MAX_VALID_LOSS = "max_valid_loss"
MIN_FRAME_ITEMS = "min_frame_items"
MIN_VALID_EPOCHS = "min_valid_epochs"
NORM_EMBED_VECS = "norm_embed_vecs"
NUM_LENGTH_FRAMES = "num_length_frames"
OPTIMIZER_NAME = "optimizer_func"
PAD_LABEL = "pad_label"
RANDOM_SAMPLING = "random_sampling"
SEQUENTIAL_SAMPLING = "sequential_sampling"
SGD_OPT = 'sgd'
TEST_SET = "test_set"
TRAIN_SET = "train_set"
SAMPLING_TYPE = "sampling_type"
SEED_VALUE = "seed_value"
TEXT_COL = "text_col"
TOKENS_COL = "tokens_col"
UNK_LABEL = "unk_label"
VALIDATION_SET = "validation_set"


In [ ]:
import json
import numpy as np
from tqdm import tqdm
with open("workshop_data/nb_config.json", 'r', encoding='utf-8') as f:
    config = json.load(f)
print(config)

added_words = [PAD_LABEL, UNK_LABEL]
class Embedded_Words:
    def __init__(self, model_file: str, added_pads: list, norm: bool) -> None:
        self.vectors, self.w2i, self.i2w = self.read_model(model_file, added_pads, norm)

    def read_model(self, model_file: str, added_pads: list, norm: bool) -> tuple:
        with open(model_file, "r", encoding="utf-8") as f:
            lines = [x.strip() for x in f.readlines()]

        print(model_file)
        print(len(lines))
        print(lines[0])

        num_word, dim = [int(x) for x in lines[0].split()]
        vectors = np.zeros((num_word + len(added_pads), dim))
        w2i = {}
        i2w = {}
        for line in tqdm(lines[1:]):
            tokens = line.split()
            word = tokens[0]
            word_index = len(w2i)
            v = np.array([float(x) for x in tokens[1:]])
            if norm:
                v = v / np.linalg.norm(v)
            vectors[word_index] = v
            w2i[word] = word_index
            i2w[word_index] = word

        for word in added_pads:
            word_index = len(w2i)
            w2i[word] = word_index
            i2w[word_index] = word
        
        return vectors, w2i, i2w

embedded_words = Embedded_Words(config[EMBED_WORDS_FILE], added_words, True)
print("")
print(embedded_words.vectors.shape)
print(embedded_words.vectors[embedded_words.vectors.shape[0]-5:,:10])

{'batch_size': 64, 'cnn_kernels': [3, 4, 5], 'cnn_out_channels': 100, 'dropout': 0.5, 'early_stop_max_no_imp': 2, 'embed_words_file': 'workshop_data/embed.model', 'freeze_embeding': 1, 'learning_rate': 0.25, 'loss_function': 'cross_entropy_loss', 'log_file_name': 'workshop_data/cnn_imdb_nb.txt', 'min_valid_epochs': 10, 'max_valid_loss': 0.1, 'min_frame_items': 500, 'norm_embed_vecs': 1, 'num_epochs': 50, 'num_length_frames': 12, 'optimizer_func': 'adadelta', 'seed_value': -1, 'sampling_type': 'random_sampling', 'label_col': 'sentiment', 'length_col': 'lengths', 'text_col': 'review', 'tokens_col': 'tokens', 'traditional_feature_type': 'tfidf', 'traditional_max_features': 7500, 'train_set': 'workshop_data/imdb_train_set.csv', 'test_set': 'workshop_data/imdb_test.csv', 'validation_set': 'workshop_data/imdb_val_set.csv'}
workshop_data/embed.model
116380
116379 200


100%|██████████| 116379/116379 [00:08<00:00, 14260.07it/s]


(116381, 200)
[[-0.01921616 -0.05748059  0.08354403 -0.13734939  0.02017608  0.118749
  -0.10769267  0.09068727 -0.04097761  0.02756285]
 [-0.03311227 -0.03198076  0.09188014 -0.10075315 -0.00427596  0.0708071
  -0.08297284  0.03524867  0.02613228  0.05615779]
 [ 0.00295082 -0.09751952  0.08996978 -0.10860393 -0.00678612  0.04517275
  -0.12002896  0.08097963  0.06024373  0.05645853]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]]


In [ ]:
def get_active_device():
    """Picking GPU if available or else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
active_device = get_active_device()
print(active_device)

cuda


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class CNN(nn.Module):
    def __init__(self,
                 config: dict,
                 num_classes):
        super(CNN, self).__init__()

        print('Freeze embedding matrix = ' + str(config[FREEZE_EMBEDDING]))
        print('load embedding model')
        added_words = [PAD_LABEL, UNK_LABEL]
        self.w2v_model = Embedded_Words("workshop_data/embed.model", added_words, True)
        print('\tw2v after padding: ' + str(self.w2v_model.vectors.shape))

        print('generate embedding tensor')
        # Set the embedding module.
        self.embedding = nn.Embedding.from_pretrained(
            torch.FloatTensor(self.w2v_model.vectors).to(active_device),
            padding_idx = self.w2v_model.w2i[PAD_LABEL],
            freeze=config[FREEZE_EMBEDDING])
        
        print('allocate convolution  layers')
        filter_width = config[CNN_OUT_CHANNELS]
        kernels = config[CNN_KERNELS]
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=self.w2v_model.vectors.shape[1],
                      out_channels=filter_width,
                      kernel_size=kernel).to(active_device)
            for kernel in kernels
        ])

        self.fc = nn.Linear(filter_width * len(kernels), num_classes).to(active_device)
        self.dropout = nn.Dropout(p=config[DROPOUT])

    def forward(self, input_ids):
        # input_ids = [#batch size, sentence len]
        x_embed = self.embedding(input_ids).float()
        #x_embed = [batch size, sentence len, embed dim]
        
        x_reshape = x_embed.permute(0, 2, 1)
        # x_reshape = [batch size, embed dim, sentence len]
        
        x_conv_list = [F.relu(conv(x_reshape)) for conv in self.convs]
        # x_conv = [batch size, out_channel_width, sentence len - kernel size + 1]
        
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2]) for x_conv in x_conv_list]
        # x_pool = [batch size, out_channel_width, 1]
        
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list], dim=1)
        # x_fc = [batch size, out_channel_width * #filters]
        
        logits = self.fc(self.dropout(x_fc))
        # logits = [batch size, #classes]
        
        return logits
    
    def get_w2v_model(self):
        return self.w2v_model


In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import RandomSampler
from torch.utils.data import SequentialSampler
from torch.utils.data import TensorDataset

def break_df_by_len(df: pd.DataFrame, config: dict) -> list:
    df[config[TOKENS_COL]] = df[config[TEXT_COL]].apply(lambda x: x.split(' '))
    df[config[LENGTH_COL]] = df[config[TOKENS_COL]].map(len)
    occurrence2len = dict(df[config[LENGTH_COL]].value_counts())
    occurrences = sorted(occurrence2len.items())
    frame_len = int(len(occurrences) / config[NUM_LENGTH_FRAMES]) + 1
    
    df_list = []
    start_frame = 0
    end_frame = frame_len
    while start_frame < len(occurrences):
        total_occurs, end_frame = get_end_frame(start_frame, end_frame, occurrences, config)
            
        if end_frame > len(occurrences):
            end_frame = len(occurrences)
            
        curr_df = df[df[config[LENGTH_COL]] >= occurrences[start_frame][0]]
        curr_df = curr_df[curr_df[config[LENGTH_COL]] <= occurrences[end_frame - 1][0]]
        
        df_list.append(curr_df)
        
        start_frame = end_frame
        end_frame += frame_len
        
    total_occurs = 0
    for curr_df in df_list:
        total_occurs += len(curr_df)
    return df_list

def get_end_frame(prev_start: int, prev_end: int, occurrences: list, config: dict) -> tuple:
    total_occurs = sum([x[1] for x in occurrences[prev_start:prev_end]])
    
    if total_occurs >= config[MIN_FRAME_ITEMS]:
        return total_occurs, prev_end
    
    total_left = sum([x[1] for x in occurrences[prev_start:]])
    if total_left < 2 * config[MIN_FRAME_ITEMS]:
        end_frame = len(occurrences)
        return total_left, end_frame
    
    end_frame = prev_end
    while total_occurs < config[MIN_FRAME_ITEMS]:
        total_occurs += occurrences[end_frame][1]
        end_frame += 1
        
    return total_occurs, (end_frame + 1)

def df_to_dataloader(df: pd.DataFrame, w2v_model: Embedded_Words, config: dict, sampling_type: str) -> DataLoader:
    texts = df[config[TOKENS_COL]].values.tolist()
    labels = df[config[LABEL_COL]].values.tolist()
    max_len = df[config[TOKENS_COL]].map(len).max()

    indexed_texts = []
    for sentence in texts:
        sentence += [PAD_LABEL] * (max_len - len(sentence))
        
        ids = [w2v_model.w2i[UNK_LABEL] if not word in w2v_model.w2i else w2v_model.w2i[word] for word in sentence]
        indexed_texts.append(ids)
        
    inputs, labels = tuple(torch.tensor(data) for data in [indexed_texts, labels])
    
    data = TensorDataset(inputs, labels)
    
    if sampling_type == RANDOM_SAMPLING:
        sampler = RandomSampler(data)
    elif sampling_type == SEQUENTIAL_SAMPLING:
        sampler = SequentialSampler(data)
    else:
        print('Wrong Sampling Type: ' + sampling_type)
        return None
        
    dataloader = DataLoader(data, sampler=sampler, batch_size=config[BATCH_SIZE])
    return dataloader
    

def get_data_loaders(input_df: pd.DataFrame, w2v_model: Embedded_Words, config: dict, sampling_type: str) -> list:
    df_list = break_df_by_len(input_df, config)
    dataloaders = []
    for df in df_list:
        dataloader = df_to_dataloader(df, w2v_model, config, sampling_type)
        dataloaders.append(dataloader)
        
    return dataloaders

RHO = 0.95
import torch.optim as optim
def get_optimizer(opt_name: str, parameters, lr):
    optimizer = None
    if opt_name == ADADELATA_OPT:
        optimizer = optim.Adadelta(parameters,
                                   lr=lr,
                                   rho=RHO)
    elif opt_name == SGD_OPT:
        optimizer = optim.SGD(parameters, lr)
    elif opt_name == ADAM_OPT:
        optimizer = optim.Adam(parameters, lr=lr)
    else:
        print('Wrong optimizer name: ' + opt_name)
        
    return optimizer
    
def get_loss_function(func_name: str):
    loss_func = None
    if func_name == CROSS_ENTROP_LOSS:
        loss_func = nn.CrossEntropyLoss()
    elif func_name == BCE_LOSS:
        loss_func = nn.BCELoss()
    else:
        print('Wrong loss function name: ' + func_name)
        
    return loss_func



In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import time

def cnn_test(model: nn.Module, test_dl: list) -> tuple:
    corrects = 0
    evaluated = 0
    t0 = time.time()
    model.eval()
    for dataloader in test_dl:
        for step, data in enumerate(dataloader):
            ids, labels = data
            ids = ids.to(active_device)
            labels = labels.to(active_device)
            with torch.no_grad():
                logits = model(ids)
            preds = torch.argmax(logits, dim=1)
            corrects += (preds == labels).sum().item()
            evaluated += ids.shape[0]
        
    accuracy = corrects / evaluated
    run_time = time.time() - t0
    return accuracy, run_time


In [ ]:
def end_train(last_model: nn.Module, opt_model: nn.Module, config: dict, test_dl, val_dl, log_file):
    accuracy, run_time = cnn_test(last_model, test_dl)
    str_acc = "{:.5f}".format(accuracy)
    str_time = "{:.0f}".format(run_time)
    log_file.write('Last Model\t' + str_acc + '\t' + str_time + '\n')
    log_file.flush()
    print('Last Model\t' + str_acc + '\t' + str_time)
        
    # Print optimal
    opt_acc, run_time = cnn_test(opt_model, test_dl)
    val_acc, run_time = cnn_test(opt_model, val_dl)

    test_acc = "test: {:.5f}".format(opt_acc)
    val_acc = "val: {:.5f}".format(val_acc)
    log_file.write('Optimal Model\tTest=' + test_acc + '\tVal=' + val_acc + '\n')
    log_file.flush()
    print('Optimal Model\tTest=' + test_acc + '\tVal=' + val_acc)
    log_file.write("\nConfiguraton:\n")
    sorted_params = sorted(config.items())
    for param in sorted_params:
        log_file.write(param[0] + "\t" + str(param[1]) + "\n")
        log_file.flush()

import random
def set_seed(seed_value: int):
    if seed_value >= 0:
        random.seed(seed_value)
        np.random.seed(seed_value)
        torch.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)

import copy
NUM_EPOCHS = 100
class Trainer:
    def __init__(self):
        pass

    def train(self, config: dict) -> tuple:
        print('cnn trainer - start')
        log_file = open(config[LOG_FILE_NAME], "w", encoding="utf-8")
        seed_value = config[SEED_VALUE]
        set_seed(seed_value)

        train_df = pd.read_csv(config[TRAIN_SET])
        val_df = pd.read_csv(config[VALIDATION_SET])
        test_df = pd.read_csv(config[TEST_SET])
        pending_model = CNN(config, train_df[config[LABEL_COL]].unique().shape[0])
        optimal_model = None

        train_dl = get_data_loaders(train_df, pending_model.get_w2v_model(), config, config[SAMPLING_TYPE])
        val_dl = get_data_loaders(val_df, pending_model.get_w2v_model(), config, config[SAMPLING_TYPE])
        test_dl = get_data_loaders(test_df, pending_model.get_w2v_model(), config, config[SAMPLING_TYPE])
        
        print('set optimizer & loss')
        best_val_acc = 0
        best_epoch = -1
        optimizer = get_optimizer(config[OPTIMIZER_NAME], pending_model.parameters(), config[LEARNING_RATE])
        loss_func = get_loss_function(config[LOSS_FUNCTION])
        
        print('get data loaders')
        sampling = RANDOM_SAMPLING if seed_value < 0 else SEQUENTIAL_SAMPLING
        print('\tsampling = ' + sampling)
        
        num_epochs = NUM_EPOCHS
        print('start training loops. #epochs = ' + str(num_epochs))
        print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Train Acc':^9} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*30)  
        
        log_file.write(f"{'Epoch':^7} | {'Train Loss':^12} | {'Train Acc':^9} | {'Val Acc':^9} | {'Elapsed':^9}\n")
        log_file.write("-"*30 + '\n')  
        log_file.flush()
            
        
        min_loss = 100
        num_no_imp = 0
        for i in range(num_epochs):
            epoch = i + 1
            epoch_start_time = time.time()
            total_loss = 0
            num_batches = 0
            
            pending_model.train()
            
            for dataloader in train_dl:
                for step, batch in enumerate(dataloader):
                    ids, labels = tuple(t for t in batch)
                    ids = ids.to(active_device)
                    labels = labels.to(active_device)
                    
                    optimizer.zero_grad()
                    
                    logits = pending_model(ids)
                    
                    loss = loss_func(logits, labels)
                    total_loss += loss.item()
                    num_batches += 1
                    
                    loss.backward()
                    
                    optimizer.step()
                
            avg_loss = total_loss / num_batches
            epoch_time = time.time() - epoch_start_time
            
            # Validation test.
            val_acc, val_time = cnn_test(pending_model, val_dl)
            train_acc, val_time = cnn_test(pending_model, train_dl)
            val_acc *= 100
            train_acc *= 100
            print(f"{epoch:^7} | {avg_loss:^12.6f} | {train_acc:^9.2f} | {val_acc:^9.2f} | {epoch_time:^9.2f}")
            log_file.write(f"{epoch:^7} | {avg_loss:^12.6f} | {train_acc:^9.2f} | {val_acc:^9.2f} | {epoch_time:^9.2f}\n")
            log_file.flush()
                
            if avg_loss < min_loss:
                min_loss = avg_loss
                num_no_imp = 0
            else:
                num_no_imp += 1
                
            if num_no_imp > config[EARLY_STOP_MAX_NO_IMP]:
                print('early stop exit')
                log_file.write('\tEarly Stop exit\n')
                log_file.flush()
                break
            
            if epoch < config[MIN_VALID_EPOCHS]:
                continue
            
            if avg_loss > config[MAX_VALID_LOSS]:
                continue
            
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                optimal_model = copy.deepcopy(pending_model)
                best_epoch = epoch
        
        print('train_cnn_nlp - end')
        end_train(pending_model, optimal_model, config, test_dl, val_dl, log_file)
        log_file.close()
        return pending_model, optimal_model, best_epoch


In [ ]:
with open("workshop_data/nb_config.json", "r", encoding="utf-8") as f:
  actual_config = json.load(f)

trainer = Trainer()
last_model, opt_model, best_epoch = trainer.train(actual_config)
print(best_epoch)

cnn trainer - start
Freeze embedding matrix = 1
load embedding model
workshop_data/embed.model
116380
116379 200


100%|██████████| 116379/116379 [00:09<00:00, 12443.28it/s]


	w2v after padding: (116381, 200)
generate embedding tensor
allocate convolution  layers
set optimizer & loss
get data loaders
	sampling = random_sampling
start training loops. #epochs = 100
 Epoch  |  Train Loss  | Train Acc |  Val Acc  |  Elapsed 
------------------------------
   1    |   0.686667   |   50.26   |   48.55   |   0.42   
   2    |   0.662800   |   74.75   |   69.93   |   0.40   
   3    |   0.599557   |   71.34   |   68.37   |   0.41   
   4    |   0.529423   |   79.60   |   76.61   |   0.40   
   5    |   0.483931   |   80.27   |   75.50   |   0.43   
   6    |   0.462344   |   82.06   |   76.17   |   0.40   
   7    |   0.437107   |   77.95   |   73.05   |   0.43   
   8    |   0.413158   |   83.07   |   75.72   |   0.41   
   9    |   0.396882   |   85.05   |   75.95   |   0.52   
  10    |   0.381708   |   84.82   |   75.95   |   0.56   
  11    |   0.360405   |   83.31   |   74.16   |   0.55   
  12    |   0.358750   |   86.91   |   76.17   |   0.41   
  13    |  